In [ ]:
%pip install -q langchain langchain-google-vertexai google-cloud-aiplatform google-auth pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.6/449.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 4.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.


In [ ]:
from google.colab import files
import os
import json

In [ ]:
print("Please upload your GCP service-account JSON key file (recommended: a project-level service account with Vertex AI permissions).")
uploaded = files.upload()

Please upload your GCP service-account JSON key file (recommended: a project-level service account with Vertex AI permissions).


Saving food-donation-agent-76f6c6f468d6.json to food-donation-agent-76f6c6f468d6 (2).json


In [ ]:
if len(uploaded) == 0:
    raise SystemExit("No file uploaded. Upload service account JSON and re-run this cell.")

In [ ]:
sa_file = list(uploaded.keys())[0]
SA_KEY_PATH = f"/content/{sa_file}"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = SA_KEY_PATH
print("Set GOOGLE_APPLICATION_CREDENTIALS =", SA_KEY_PATH)

Set GOOGLE_APPLICATION_CREDENTIALS = /content/food-donation-agent-76f6c6f468d6 (2).json


In [ ]:
PROJECT_ID = input("Enter your GCP project id (e.g. my-project-123): ").strip()
REGION = input("Enter Vertex AI region (e.g. us-central1) [default us-central1]: ").strip() or "us-central1"

Enter your GCP project id (e.g. my-project-123): food-donation-agent
Enter Vertex AI region (e.g. us-central1) [default us-central1]: us-central1


In [ ]:
os.environ["GCLOUD_PROJECT"] = PROJECT_ID
os.environ["GCP_PROJECT"] = PROJECT_ID

In [ ]:
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=REGION)
print("Vertex AI initialized for project:", PROJECT_ID, "region:", REGION)

Vertex AI initialized for project: food-donation-agent region: us-central1


In [ ]:
# ===============================
# 4️⃣ Import LangChain + VertexAI LLM
# ===============================
try:
    from langchain_google_vertexai import VertexAI
except Exception:
    from langchain_google_vertexai.llms import VertexAI

model_name = input("Vertex model name (default text-bison@001): ").strip() or "text-bison@001"
llm = VertexAI(model_name=model_name)
print("LLM wrapper created for model:", model_name)

Vertex model name (default text-bison@001): gemini-2.5-pro
LLM wrapper created for model: gemini-2.5-pro


In [ ]:
# ===============================
# 5️⃣ Global "database" dictionary
# ===============================
donation_entries = {}

In [ ]:
import uuid
from datetime import datetime, timedelta
import re

In [ ]:
# ===============================
# 6️⃣ Define donation intake tool
# ===============================
try:
    from langchain_core.tools import tool as tool_decorator
except Exception:
    from langchain.tools import tool as tool_decorator

In [ ]:
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field

class DonationDetails(BaseModel):
    item_name: str = Field(..., description="Name of the donated food item")
    quantity: int = Field(..., description="Quantity donated")
    unit: str = Field(..., description="Unit of measurement such as kg, liters, pcs")
    expiry_days: int = Field(..., description="Days remaining before expiry")

In [ ]:
parser = PydanticOutputParser(pydantic_object=DonationDetails)

prompt = PromptTemplate(
    template=(
        "Extract structured donation details from the following message:\n"
        "{text}\n\n"
        "{format_instructions}"
    ),
    input_variables=["text"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [ ]:
@tool_decorator("donation_intake", return_direct=True)
def donation_intake(text: str) -> str:
    """Extracts and stores donation details using Vertex AI structured output."""
    global donation_entries

    # Build extraction chain
    chain = prompt | llm | parser

    try:
        details = chain.invoke({"text": text})
    except Exception as e:
        return f"Could not parse donation: {str(e)}"

    # Compute expiry date
    expiry_date = datetime.now() + timedelta(days=details.expiry_days)
    expiry_date_str = expiry_date.strftime("%Y-%m-%d")
    days_left = details.expiry_days

    # Urgency and status calculation (new logic)
    if days_left <= 0:
        urgency_score, status = 0, "expired"
    elif days_left == 1:
        urgency_score, status = 5, "available"
    elif 2 <= days_left <= 3:
        urgency_score, status = 4, "available"
    elif 4 <= days_left <= 5:
        urgency_score, status = 3, "available"
    elif 6 <= days_left <= 7:
        urgency_score, status = 2, "available"
    else:
        urgency_score, status = 1, "available"

    # Assign unique entry ID
    entry_id = str(uuid.uuid4())

    # Store in global dict
    donation_entries[entry_id] = {
        "entry_id": entry_id,
        "item_name": details.item_name,
        "quantity": details.quantity,
        "unit": details.unit,
        "expiry_date": expiry_date_str,
        "days_left": days_left,
        "urgency_score": urgency_score,
        "status": status,
    }

    # Response
    return (
        f"Donation registered! ID: {entry_id}, "
        f"Item: {details.item_name}, Quantity: {details.quantity} {details.unit}, "
        f"Expiry in {days_left} days (Urgency: {urgency_score}/5)."
    )

print("Donation Intake tool ready!")

Donation Intake tool ready!


In [ ]:
donation_entries = {}

In [ ]:
# Install dependencies
%pip install -q streamlit pyngrok


Overwriting donation_agent_app.py


In [ ]:
%%writefile donation_agent_app.py
import streamlit as st
import uuid
from datetime import datetime, timedelta

# ========== LangChain & VertexAI setup ==========
try:
    from langchain_google_vertexai import VertexAI
except Exception:
    from langchain_google_vertexai.llms import VertexAI

from google.cloud import aiplatform
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langchain_core.tools import tool as tool_decorator
from langchain.agents import initialize_agent, AgentType

# ========== Vertex AI Initialization ==========
PROJECT_ID = "food-donation-agent"
REGION = "us-central1"
aiplatform.init(project=PROJECT_ID, location=REGION)

# ========== Vertex AI Model ==========
llm = VertexAI(model_name="gemini-2.5-pro")

# ========== Global store ==========
if "donation_entries" not in st.session_state:
    st.session_state.donation_entries = {}

# Initialize theme
if "theme" not in st.session_state:
    st.session_state.theme = "dark"

# ========== Data Model ==========
class DonationDetails(BaseModel):
    item_name: str = Field(..., description="Name of the donated food item")
    quantity: int = Field(..., description="Quantity donated")
    unit: str = Field(..., description="Unit of measurement such as kg, liters, pcs")
    expiry_days: int = Field(..., description="Days remaining before expiry")

# ========== Parser + Prompt ==========
parser = PydanticOutputParser(pydantic_object=DonationDetails)
prompt = PromptTemplate(
    template=(
        "Extract structured donation details from the following message:\n"
        "{text}\n\n"
        "{format_instructions}"
    ),
    input_variables=["text"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# ========== Tool ==========
@tool_decorator(
    "donation_intake",
    description="Extracts and stores donation details using Vertex AI structured output",
    return_direct=True
)
def donation_intake(text: str) -> str:
    chain = prompt | llm | parser
    try:
        details = chain.invoke({"text": text})
    except Exception as e:
        return f"Could not parse donation: {str(e)}"

    expiry_date = datetime.now() + timedelta(days=details.expiry_days)
    expiry_date_str = expiry_date.strftime("%Y-%m-%d")
    days_left = details.expiry_days

    if days_left <= 0:
        urgency_score, status = 0, "expired"
    elif days_left == 1:
        urgency_score, status = 5, "available"
    elif 2 <= days_left <= 3:
        urgency_score, status = 4, "available"
    elif 4 <= days_left <= 5:
        urgency_score, status = 3, "available"
    elif 6 <= days_left <= 7:
        urgency_score, status = 2, "available"
    else:
        urgency_score, status = 1, "available"

    entry_id = str(uuid.uuid4())
    st.session_state.donation_entries[entry_id] = {
        "entry_id": entry_id,
        "item_name": details.item_name,
        "quantity": details.quantity,
        "unit": details.unit,
        "expiry_date": expiry_date_str,
        "days_left": days_left,
        "urgency_score": urgency_score,
        "status": status,
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

    return (
        f"✅ Donation registered!\n\n**ID:** {entry_id}\n"
        f"**Item:** {details.item_name}\n"
        f"**Quantity:** {details.quantity} {details.unit}\n"
        f"**Expiry in:** {days_left} days (Urgency: {urgency_score}/5)"
    )

# ========== Agent ==========
agent = initialize_agent(
    tools=[donation_intake],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False,
)

# ========== Streamlit App ==========
st.set_page_config(page_title="Food Donation Agent", page_icon="🍽️", layout="centered")

# Initialize session state for page navigation
if "page" not in st.session_state:
    st.session_state.page = "home"

# ========== THEME STYLES ==========
def get_theme_styles():
    if st.session_state.theme == "dark":
        return {
            "bg_primary": "#0d1117",
            "bg_secondary": "#161b22",
            "bg_tertiary": "#21262d",
            "text_primary": "#e6edf3",
            "text_secondary": "#8b949e",
            "border_color": "#30363d",
            "border_hover": "#444c56",
            "input_bg": "#21262d",
            "input_focus_bg": "#2d333b",
            "shadow": "rgba(0, 0, 0, 0.3)",
            "user_msg_bg": "#2d333b",
            "bot_msg_bg": "#21262d",
        }
    else:  # light mode
        return {
            "bg_primary": "#ffffff",
            "bg_secondary": "#f6f8fa",
            "bg_tertiary": "#ffffff",
            "text_primary": "#1f2328",
            "text_secondary": "#656d76",
            "border_color": "#d0d7de",
            "border_hover": "#a6b2c0",
            "input_bg": "#f6f8fa",
            "input_focus_bg": "#ffffff",
            "shadow": "rgba(0, 0, 0, 0.1)",
            "user_msg_bg": "#ddf4ff",
            "bot_msg_bg": "#f6f8fa",
        }

# ========== THEME TOGGLE ==========
def theme_toggle_button():
    """Renders just the theme toggle button"""
    if st.session_state.theme == "dark":
        if st.button("☀️", key="theme_btn", help="Switch to Light Mode"):
            st.session_state.theme = "light"
            st.rerun()
    else:
        if st.button("🌙", key="theme_btn", help="Switch to Dark Mode"):
            st.session_state.theme = "dark"
            st.rerun()

# ========== HOME PAGE ==========
def show_home_page():
    colors = get_theme_styles()

    st.markdown(f"""
    <style>
    .stApp {{
        background-color: {colors['bg_primary']};
    }}

    .home-container {{
        display: flex;
        flex-direction: column;
        align-items: center;
        justify-content: center;
        min-height: 70vh;
        text-align: center;
    }}

    .home-title {{
        font-size: 3.5rem;
        font-weight: bold;
        color: {colors['text_primary']};
        margin-bottom: 1rem;
    }}

    .home-subtitle {{
        font-size: 1.3rem;
        color: {colors['text_secondary']};
        margin-bottom: 3rem;
        max-width: 600px;
    }}

    .feature-box {{
        background-color: {colors['bg_secondary']};
        border: 1px solid {colors['border_color']};
        border-radius: 12px;
        padding: 1.5rem;
        margin: 1rem 0;
        max-width: 500px;
    }}

    .feature-title {{
        color: {colors['text_primary']};
        font-size: 1.2rem;
        font-weight: bold;
        margin-bottom: 0.5rem;
    }}

    .feature-text {{
        color: {colors['text_secondary']};
        font-size: 1rem;
    }}

    .stButton>button {{
        background-color: #238636 !important;
        color: white !important;
        border: none !important;
        border-radius: 8px !important;
        padding: 0.75rem 2rem !important;
        font-size: 1.1rem !important;
        font-weight: bold !important;
        cursor: pointer !important;
        transition: all 0.3s ease !important;
    }}

    .stButton>button:hover {{
        background-color: #2ea043 !important;
        transform: translateY(-2px);
        box-shadow: 0 4px 12px rgba(35, 134, 54, 0.4) !important;
    }}

    div[data-testid="stButton"] button[kind="secondary"] {{
        background-color: transparent !important;
        color: {colors['text_primary']} !important;
        border: 1px solid {colors['border_color']} !important;
        font-size: 1.5rem !important;
        padding: 0.3rem 0.6rem !important;
    }}

    div[data-testid="stButton"] button[kind="secondary"]:hover {{
        background-color: {colors['bg_secondary']} !important;
        border-color: {colors['border_hover']} !important;
        transform: none !important;
        box-shadow: none !important;
    }}

    .title-row {{
        display: flex;
        justify-content: flex-end;
        align-items: center;
        margin-bottom: 2rem;
    }}
    </style>
    """, unsafe_allow_html=True)

    # Theme toggle at top right
    col1, col2 = st.columns([10, 1])
    with col2:
        theme_toggle_button()

    # st.markdown('<div class="home-container">', unsafe_allow_html=True)

    title_color = '#000000' if st.session_state.theme == 'light' else colors['text_primary']
    st.markdown(f'<h1 style="font-size: 3.5rem; font-weight: bold; color: {title_color}; margin-bottom: 1rem;">🍽️ Food Donation Agent</h1>', unsafe_allow_html=True)
    st.markdown('<p class="home-subtitle">Connect food donors with those in need using AI-powered intelligent assistance</p>', unsafe_allow_html=True)

    col1, col2, col3 = st.columns(3)

    with col1:
        st.markdown(f"""
        <div class="feature-box">
            <div class="feature-title">🤖 AI-Powered</div>
            <div class="feature-text">Smart extraction of donation details</div>
        </div>
        """, unsafe_allow_html=True)

    with col2:
        st.markdown(f"""
        <div class="feature-box">
            <div class="feature-title">⚡ Easy</div>
            <div class="feature-text">Simple chat interface for donations</div>
        </div>
        """, unsafe_allow_html=True)

    with col3:
        st.markdown(f"""
        <div class="feature-box">
            <div class="feature-title">📊 Track Urgency</div>
            <div class="feature-text">Automatic urgency scoring system</div>
        </div>
        """, unsafe_allow_html=True)

    st.markdown('</div>', unsafe_allow_html=True)

    col1, col2, col3 = st.columns([1, 1, 1])

    with col1:
        if st.button("🚀 Start Donating", use_container_width=True):
            st.session_state.page = "chat"
            st.rerun()

    with col3:
        if st.button("📋 My Donations", use_container_width=True):
            st.session_state.page = "donations"
            st.rerun()

# ========== DONATIONS PAGE ==========
def show_donations_page():
    colors = get_theme_styles()

    st.markdown(f"""
    <style>
    .stApp {{
        background-color: {colors['bg_primary']};
    }}

    .block-container {{
        padding-top: 4rem !important;
    }}

    h1 {{
        color: {colors['text_primary']} !important;
    }}

    .donation-card {{
        background-color: {colors['bg_secondary']};
        border: 1px solid {colors['border_color']};
        border-radius: 12px;
        padding: 1.5rem;
        margin-bottom: 1rem;
        box-shadow: 0 2px 4px {colors['shadow']};
    }}

    .donation-header {{
        display: flex;
        justify-content: space-between;
        align-items: center;
        margin-bottom: 1rem;
        padding-bottom: 0.5rem;
        border-bottom: 1px solid {colors['border_color']};
    }}

    .donation-item {{
        color: {colors['text_primary']};
        font-size: 1.3rem;
        font-weight: bold;
    }}

    .urgency-badge {{
        padding: 0.3rem 0.8rem;
        border-radius: 12px;
        font-size: 0.85rem;
        font-weight: bold;
    }}

    .urgency-5 {{ background-color: #da3633; color: white; }}
    .urgency-4 {{ background-color: #f85149; color: white; }}
    .urgency-3 {{ background-color: #d29922; color: white; }}
    .urgency-2 {{ background-color: #58a6ff; color: white; }}
    .urgency-1 {{ background-color: #3fb950; color: white; }}
    .urgency-0 {{ background-color: #8b949e; color: white; }}

    .donation-details {{
        color: {colors['text_secondary']};
        font-size: 0.95rem;
        line-height: 1.8;
    }}

    .donation-details strong {{
        color: {colors['text_primary']};
    }}

    .no-donations {{
        text-align: center;
        padding: 3rem;
        color: {colors['text_secondary']};
        font-size: 1.1rem;
    }}

    .stButton>button {{
        background-color: {colors['bg_tertiary']} !important;
        color: {colors['text_primary']} !important;
        border: 1px solid {colors['border_color']} !important;
        border-radius: 8px !important;
        padding: 0.5rem 1rem !important;
        font-size: 0.9rem !important;
    }}

    .stButton>button:hover {{
        background-color: {colors['bg_secondary']} !important;
        border-color: {colors['border_hover']} !important;
    }}

    div[data-testid="stButton"] button[kind="secondary"] {{
        background-color: transparent !important;
        color: {colors['text_primary']} !important;
        border: 1px solid {colors['border_color']} !important;
        font-size: 1.5rem !important;
        padding: 0.3rem 0.6rem !important;
    }}

    div[data-testid="stButton"] button[kind="secondary"]:hover {{
        background-color: {colors['bg_secondary']} !important;
        border-color: {colors['border_hover']} !important;
    }}
    </style>
    """, unsafe_allow_html=True)

    col1, col2, col3 = st.columns([6, 2, 1])
    with col1:
        if st.button("← Back to Home"):
            st.session_state.page = "home"
            st.rerun()
    with col2:
        if st.button("🚀 Start Donating"):
            st.session_state.page = "chat"
            st.rerun()

    # Title with theme toggle on the same row
    title_col, toggle_col = st.columns([10, 1])
    with title_col:
        st.title("📋 My Donations")
    with toggle_col:
        theme_toggle_button()

    if not st.session_state.donation_entries:
        st.markdown("""
        <div class="no-donations">
            <h2>🍽️</h2>
            <p>No donations yet. Start donating to make a difference!</p>
        </div>
        """, unsafe_allow_html=True)
    else:
        st.markdown(f"<p style='color: {colors['text_secondary']}; margin-bottom: 1.5rem;'>Total Donations: {len(st.session_state.donation_entries)}</p>", unsafe_allow_html=True)

        sorted_donations = sorted(
            st.session_state.donation_entries.values(),
            key=lambda x: x['urgency_score'],
            reverse=True
        )

        for donation in sorted_donations:
            urgency_class = f"urgency-{donation['urgency_score']}"
            status_emoji = "❌" if donation['status'] == "expired" else "✅"

            st.markdown(f"""
            <div class="donation-card">
                <div class="donation-header">
                    <div class="donation-item">{donation['item_name']}</div>
                    <div class="urgency-badge {urgency_class}">Urgency: {donation['urgency_score']}/5</div>
                </div>
                <div class="donation-details">
                    <strong>Quantity:</strong> {donation['quantity']} {donation['unit']}<br>
                    <strong>Expiry Date:</strong> {donation['expiry_date']} ({donation['days_left']} days left)<br>
                    <strong>Status:</strong> {status_emoji} {donation['status'].title()}<br>
                    <strong>Donated on:</strong> {donation['timestamp']}<br>
                    <strong>ID:</strong> <code style="background-color: {colors['bg_tertiary']}; padding: 0.2rem 0.4rem; border-radius: 4px; font-size: 0.85rem;">{donation['entry_id']}</code>
                </div>
            </div>
            """, unsafe_allow_html=True)

# ========== CHAT PAGE ==========
def show_chat_page():
    colors = get_theme_styles()

    st.markdown(f"""
    <style>
    .stApp {{
        background-color: {colors['bg_primary']};
    }}

    .block-container {{
        padding-top: 2.5rem !important;
        padding-bottom: 0 !important;
    }}

    .chat-container {{
        max-height: 60vh;
        overflow-y: auto;
        padding: 1.5rem;
        border-radius: 12px;
        background-color: {colors['bg_secondary']};
        margin-top: 2rem;
        margin-bottom: 120px;
        border: 1px solid {colors['border_color']};
        box-shadow: 0 4px 6px {colors['shadow']};
    }}

    .chat-container::-webkit-scrollbar {{
        width: 8px;
    }}

    .chat-container::-webkit-scrollbar-track {{
        background: {colors['bg_primary']};
        border-radius: 10px;
    }}

    .chat-container::-webkit-scrollbar-thumb {{
        background: {colors['border_color']};
        border-radius: 10px;
    }}

    .chat-container::-webkit-scrollbar-thumb:hover {{
        background: {colors['border_hover']};
    }}

    .message-wrapper {{
        display: flex;
        align-items: flex-start;
        margin-bottom: 1rem;
        clear: both;
    }}

    .avatar {{
        width: 32px;
        height: 32px;
        border-radius: 50%;
        display: flex;
        align-items: center;
        justify-content: center;
        font-size: 14px;
        font-weight: bold;
        flex-shrink: 0;
        margin-right: 10px;
    }}

    .user-avatar {{
        background-color: #dc3545;
        color: white;
    }}

    .bot-avatar {{
        background-color: #ffc107;
        color: #0d1117;
    }}

    .user-msg {{
        background-color: {colors['user_msg_bg']};
        color: {colors['text_primary']};
        padding: 12px 16px;
        border-radius: 12px;
        max-width: 70%;
        word-wrap: break-word;
        border: 1px solid {colors['border_color']};
        line-height: 1.5;
    }}

    .bot-msg {{
        background-color: {colors['bot_msg_bg']};
        color: {colors['text_primary']};
        padding: 12px 16px;
        border-radius: 12px;
        max-width: 70%;
        word-wrap: break-word;
        border: 1px solid {colors['border_color']};
        line-height: 1.5;
    }}

    # .input-area {{
    #     position: fixed;
    #     bottom: 0;
    #     left: 0;
    #     right: 0;
    #     background-color: {colors['bg_secondary']};
    #     padding: 1.5rem;
    #     border-top: 2px solid {colors['border_color']};
    #     z-index: 1000;
    #     box-shadow: 0 -4px 6px {colors['shadow']};
    # }}

    .stTextInput>div>div>input {{
        background-color: {colors['input_bg']} !important;
        color: {colors['text_primary']} !important;
        border: 2px solid {colors['border_color']} !important;
        border-radius: 10px !important;
        padding: 12px 16px !important;
        font-size: 15px !important;
    }}

    .stTextInput>div>div>input:focus {{
        border-color: #58a6ff !important;
        box-shadow: 0 0 0 3px rgba(88, 166, 255, 0.2) !important;
        background-color: {colors['input_focus_bg']} !important;
    }}

    .stTextInput>div>div>input::placeholder {{
        color: {colors['text_secondary']} !important;
    }}

    h1 {{
        color: {colors['text_primary']} !important;
        margin-top: 0 !important;
        margin-bottom: 1rem !important;
        padding-top: 0 !important;
        padding-bottom: 0 !important;
    }}

    .stButton>button {{
        background-color: {colors['bg_tertiary']} !important;
        color: {colors['text_primary']} !important;
        border: 1px solid {colors['border_color']} !important;
        border-radius: 8px !important;
        padding: 0.5rem 1rem !important;
        font-size: 0.9rem !important;
    }}

    .stButton>button:hover {{
        background-color: {colors['bg_secondary']} !important;
        border-color: {colors['border_hover']} !important;
    }}

    div[data-testid="stButton"] button[kind="secondary"] {{
        background-color: transparent !important;
        color: {colors['text_primary']} !important;
        border: 1px solid {colors['border_color']} !important;
        font-size: 1.5rem !important;
        padding: 0.3rem 0.6rem !important;
    }}

    div[data-testid="stButton"] button[kind="secondary"]:hover {{
        background-color: {colors['bg_secondary']} !important;
        border-color: {colors['border_hover']} !important;
    }}
    </style>
    """, unsafe_allow_html=True)

    col1, col2, col3 = st.columns([6, 2, 1])
    with col1:
        if st.button("← Back to Home"):
            if "messages" in st.session_state:
                del st.session_state.messages
            st.session_state.page = "home"
            st.rerun()
    with col2:
        if st.button("📋 My Donations"):
            st.session_state.page = "donations"
            st.rerun()

    # Title with theme toggle on the same row
    title_col, toggle_col = st.columns([10, 1])
    with title_col:
        st.title("🍽️ Food Donation Chat Agent")
    with toggle_col:
        theme_toggle_button()

    if "messages" not in st.session_state:
        st.session_state.messages = [
            {"role": "agent", "text": "Hi there! 👋 I'm your Food Donation Agent. Tell me what food item you'd like to donate."}
        ]

    # st.markdown('<div class="chat-container" id="chat-messages">', unsafe_allow_html=True)
    for msg in st.session_state.messages:
        if msg["role"] == "user":
            st.markdown(f'''
            <div class="message-wrapper">
                <div class="avatar user-avatar">U</div>
                <div class="user-msg">{msg["text"]}</div>
            </div>
            ''', unsafe_allow_html=True)
        else:
            st.markdown(f'''
            <div class="message-wrapper">
                <div class="avatar bot-avatar">A</div>
                <div class="bot-msg">{msg["text"]}</div>
            </div>
            ''', unsafe_allow_html=True)
    st.markdown('</div>', unsafe_allow_html=True)

    # Auto-scroll to bottom
    st.markdown("""
    <script>
        const chatContainer = document.getElementById('chat-messages');
        if (chatContainer) {
            chatContainer.scrollTop = chatContainer.scrollHeight;
        }
    </script>
    """, unsafe_allow_html=True)

    def handle_input():
        user_input = st.session_state.user_input
        if user_input:
            st.session_state.messages.append({"role": "user", "text": user_input})
            try:
                response = agent.run(user_input)
            except Exception as e:
                response = f"⚠️ Agent error: {e}"
            st.session_state.messages.append({"role": "agent", "text": response})
            st.session_state.user_input = ""

    st.markdown('<div class="input-area">', unsafe_allow_html=True)
    st.text_input("Type your message...", key="user_input", on_change=handle_input, label_visibility="collapsed")
    st.markdown('</div>', unsafe_allow_html=True)

# ========== MAIN APP LOGIC ==========
if st.session_state.page == "home":
    show_home_page()
elif st.session_state.page == "donations":
    show_donations_page()
else:
    show_chat_page()

Overwriting donation_agent_app.py


In [ ]:

!streamlit run donation_agent_app.py --server.port 8501 &>/dev/null &

In [ ]:
!pkill -f ngrok

In [ ]:
from pyngrok import ngrok

# Start ngrok tunnel to Streamlit port 8501
public_url = ngrok.connect(8501)
print("Your Streamlit public URL:", public_url)


Your Streamlit public URL: NgrokTunnel: "https://brimless-constance-toilsome.ngrok-free.dev" -> "http://localhost:8501"


In [ ]:
!pip install pyngrok

In [ ]:
!ngrok authtoken 33pqeJzXdZULVe4r9phJR5h9N3x_75tNfeCgSnLh1GRxVWMfL

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
